In [ ]:
from kamiwaza.services.models.services import ModelService
from kamiwaza.services.models.services import ModelFileService

# and lower our debug output
import logging
logging.basicConfig(level=logging.WARNING)

# be less noisy
logging.getLogger("requests").setLevel(logging.WARNING)
for logger_name in logging.root.manager.loggerDict:
    logger = logging.getLogger(logger_name)
    logger.setLevel(logging.WARNING)

## Getting a Model

Important note: many of Kamiwaza's services are exposed as REST endpoints, allowing you to build applications that leverage Kamiwaza features from other systems.

Most of the functions of ModelService are available in the /models namespace. See the REST docs. (Link in the GUI to full swagger spec)

In [ ]:
model_service = ModelService()
model_file_service = ModelFileService()

In [ ]:
# this doesn't have to be so verbose, but we know what we want
import platform

if platform.system() == 'Darwin':
    TARGET_MODEL = 'TheBloke/OpenHermes-2.5-Mistral-7B-GGUF'
elif platform.system() == 'Linux':
    TARGET_MODEL = 'TheBloke/OpenHermes-2.5-Mistral-7B-16k-AWQ'
else:
    raise ValueError(f"Unsupported operating system: {platform.system()}")
# search models
model_list = model_service.search_models(query=TARGET_MODEL, hubs_to_search=['*'])

# grab the first (we did not say exact=True but this should only have one result)
model = model_list[0]

# list the files associated
files = model_service.list_hub_model_files(hub=model.hub, model_id=model.repo_modelId)

# Select the ones we want - basically, one set of weights (4-bit quant), plus all the metadata/config
if platform.system() == 'Darwin':
    files_to_download = [file.name for file in files if 'gguf' not in file.name or file.name == 'openhermes-2.5-mistral-7b.Q4_K_M.gguf']
elif platform.system() == 'Linux':
    files_to_download = [file.name for file in files] # the awq version just has a single model.safetensors
else:
    raise ValueError(f"Unsupported operating system: {platform.system()}")



print(f"We will download {', '.join(files_to_download)}")

In [ ]:
# Start the download 

result = model_service.download_model_files(model_id=model.repo_modelId, hub=model.hub, files_to_download=files_to_download)
result
# Note: if you run this more than once you will get more files; once you decide to download a model,
# Kamiwaza saves all the file info to the database, even if you are opting to not download all the files immediately

In [ ]:
model_file = model_file_service.get_model_file(model_file_id=str(result['files'][0]))
model_id = model_file.m_id
print(f"{model_file.name} {model_file.dl_requested_at} {model_file.size} {model_file.m_id}")

# Background or Notebook download?

If you have the Kamiwaza service running, the downloads will be going now. We assume this is the default.

You can trigger the download manually, calling `model_service.process_downloads()`; by default, Kamiwaza does up to 6 at a time. This actually creates subprocesses, because we read the download prorgress to update internal state.

We'll assume the service is running, but process_downloads() also exits after creating download workers, so you can, if needed, add it and then continue with this code

**Note**: Kamiwaza doesn't re-download model files in general (if the checksum matches, or the size if a checksum isn't available, which is the case for small files on HF); so you can re-download this; the call will still execute, but it will evaluate checksums and then exit

In [ ]:
import time
from IPython.display import clear_output
i = 0
while True:
    downloads = model_file_service.get_download_status(model_ids=[model_id])
    if len(downloads)<1:
        break

    print('\n'.join(f"{d.name}: {d.download_percentage}% | Elapsed: {d.download_elapsed}, Remaining: {d.download_remaining}, Throughput: {d.download_throughput} | PID: {d.download_pid}" for d in downloads))
    time.sleep(2)
    clear_output(wait=True)

print("Done")

## IMPORTANT NOTE

If you are on OSX, you must have built llama.cpp with with the `bash build-llama-cpp.sh` command as per `RELEASE.md` and `kamiwaza/README.md`; if you have not, the serving service will not find llamacpp to use and this will cause an error.

(If you are on Linux your engine will be vLLM and deploy as a container. OSX does not provide Docker GPU access, so we can't go doing that.)

In [ ]:
# this is likely to change as the api should unroll a request before handing to a service

from kamiwaza.serving.schemas.serving import CreateModelDeployment
from kamiwaza.serving.services import ServingService

config = model_service.get_model_configs(model_id=model_id)[0].id

# many params have defaults/optional; the default engine on osx is llamacpp, which the installer should build
# the default on Linux is vllm although llamacpp is compatible. (As of this writing, uncertain if we will try to
# autobuild on Linux)
deployment_request = CreateModelDeployment(
    model_id = model_id,
    model_config_id = config
)

serving_service = ServingService()
deployment = serving_service.deploy_model(deployment_request)


    

## Notes

URLs here are constructed from local os reporting or ray information if ray is running

Note that the default interface on llama.cpp is the llama cpp web interface, but the OpenAI chatcompletion api endpoint is accessible on both
engines if you specify the hostname and port to the openai client. (Tested only with openai==0.28.0, the version before "conversations" were introduced)

In [ ]:
time.sleep(1)
deployments = serving_service.list_deployments(with_names=True)
if len(deployments) == 1:
    deployment_id = deployments[0].id
else:
    deployment_id = None
    
for deployment in serving_service.list_deployments(with_names=True):
    print(f"Deployment: {deployment.m_id}, Connect: http://{deployment.host_name}:{deployment.listen_port}{'/docs' if deployment.engine_name == 'vllm' else ''}, status: {deployment.status}")

## Done! Move on and come back to run this later (or use the UI)

You can move on to the inference notebook. You can run the cell below after removing the comment to programmatically stop the endpoint; or use the UI

Models->Model Details (click the model name) -> Details (on the model deployment bottom of right side column) -> Stop Deployment


In [ ]:
#if deployment_id:
#    serving_service.stop_deployment(deployment_id)
#else:
#    print("There was more than 1 deployment, so you'll have to do this yourself!")

deployments = serving_service.list_deployments(with_names=True)
if not deployments:
    print("No deployments running!")
else:
    for deployment in serving_service.list_deployments(with_names=True):
        print(f"Deployment: {deployment.m_id}, Connect: http://{deployment.host_name}:{deployment.listen_port}{'/docs' if deployment.engine_name == 'vllm' else ''}, status: {deployment.status}")

## Final Note

As a side note, the stop deployment operation is blocking; but we do set the status to **STOP_REQUESTED** before we attempt to terminate the deployment